In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
def prewhiten(x):
    """
    normalize the input
    """
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError("Dimension should be 3 or 4")

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0 / np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [ ]:
cascade = cv2.CascadeClassifier("models/haarcascade_frontalface.xml")
model = load_model("models/facenet_keras.h5")

In [ ]:
image = cv2.imread("images/test/test1.jpg")

In [ ]:
image = cv2.imread("images/test/test5.jpg")

In [ ]:
%%time

margin_half = 5

boxes = cascade.detectMultiScale(image, 1.1, 4)

aligned_images = []
for box in boxes:
    x, y, w, h = box
    cropped = image[y - margin_half:y + h + margin_half, x - margin_half:x + w + margin_half, :]
    aligned = cv2.resize(cropped, (160, 160))
    aligned_images.append(aligned)

aligned_images = np.array(aligned_images)
prewhiten_images = prewhiten(aligned_images)

In [ ]:
predicts = model.predict(prewhiten_images)

In [ ]:
# predict_on_batch khi chạy lần đầu chậm nhưng những lần sau tốc độ tốt hơn predict
# tuy nhiên nếu các batch có kích thước khác nhau thì với mỗi kích thước batch phải khởi tạo một lần
# chẳng hạn có 3 loại ảnh là ảnh có 1, 2, 3 khuôn mặt thì phải khởi tạo ba lần
predicts = model.predict_on_batch(prewhiten_images)